In [ ]:
import requests
import uuid
import time
import base64
import json
import re
from openai import OpenAI
import os

# naver ocr
def resume_ocr(image_file):
    with open(image_file,'rb') as f:
      file_data = f.read()

    request_json = {
        'images': [
            {
                'format': 'pdf',
                'name': 'demo',
                'data': base64.b64encode(file_data).decode()
            }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000))
    }

    payload = json.dumps(request_json).encode('UTF-8')
    headers = {
      'X-OCR-SECRET': ocr_secret_key,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", ocr_api_url, headers=headers, data = payload)

    res = json.loads(response.text)

    with open('result_json.json', 'w') as json_file:
      json.dump(res, json_file, indent=4, ensure_ascii=False)

    grouped_texts = {}
    # y축 오차 범위
    y_error_range = 5

    for field in res["images"][0]["fields"]:
        height = field["boundingPoly"]["vertices"][0]["y"]
        infer_text = field["inferText"]

        # 오차범위 이내의 같은 y축에 있는 값 같은 열로 취급하기
        closest_height = next((h for h in grouped_texts if abs(height - h) <= y_error_range), None)

        if closest_height is not None:
            grouped_texts[closest_height].append(infer_text)
        else:
            grouped_texts[height] = [infer_text]

    result_string = ''
    for height, texts in sorted(grouped_texts.items()):
        result_string += ' '.join(texts) + '\n'

    return result_string

# 개인 정보 가려내기
def mask_personal_info(text):
    # 예시: 전화번호, 이메일 주소를 가려냄
    phone_number_pattern = re.compile(r'\b\d{3}[-.]?\d{4}[-.]?\d{4}\b')
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    birthdate_pattern = re.compile(r'\b(\d{4})[년./\s-]?[\s]?(\d{1,2})[월./\s-]?[\s]?(\d{1,2})[일]?[생]?\b')
		# 도로명 주소, 몇동몇호만 마스킹
    # address_pattern = re.compile("(([가-힣A-Za-z·\d~\-\.]{2,}(로|길).[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d]+)")

    # 호부터 다 남아있음
    address_pattern = re.compile("(([가-힣A-Za-z·\d~\-\.]{2,}(도|시))|([가-힣A-Za-z·\d~\-\.]{2,}(시|군|구))|([가-힣A-Za-z·\d~\-\.]{2,}(읍|면))|([가-힣A-Za-z·\d~\-\.]{2,}(로|길).[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d]+)")

    # 호부터 다 없어짐 ex) 101호(명륜2가~~) -> 호(명륜2가~~) 없어짐
    # address_pattern = re.compile("(([가-힣A-Za-z·\d~\-\.]{2,}(도|시))|([가-힣A-Za-z·\d~\-\.]{2,}(시|군|구))|([가-힣A-Za-z·\d~\-\.]{2,}(읍|면))|([가-힣A-Za-z·\d~\-\.]{2,}(로|길).[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d]+.*)")


    # 개인 정보 마스킹
    masked_text = phone_number_pattern.sub('번호', text)
    masked_text = email_pattern.sub('메일', masked_text)
    #masked_text = birthdate_pattern.sub('생년월일', masked_text)
    masked_text = address_pattern.sub('주소', masked_text)

    return masked_text

def formatting_career(text):
    OpenAI.api_key = os.getenv(gpt_secret_key)
    client = OpenAI(api_key=gpt_secret_key)

    command = '경력사항을 기간, 회사명, 부서/직책명에 대한 정보를 JSON 형식으로 추출해줘. 학력사항은 기간, 학교명, 학위/전공명에 대한 정보를 JSON 형식으로 추출해줘. 기간은 기간: yyyy년 MM월-yyyy년 MM월 형식이야. 만약 없다면 빈 JSON으로 반환해줘'.'

    completion = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"{command}:\n{text}",
        temperature=0,
        max_tokens=1024,
    )

    result = completion.choices[0].text
    """
    result_json = json.loads(result)

    with open('result_json.json', 'w') as json_file:
      json.dump(result_json, json_file, indent=4, ensure_ascii=False)
    """
    return result

image_file = 'resume_sample.pdf'
ocr_result = resume_ocr(image_file)
mask_result = mask_personal_info(ocr_result)
formatting_result = formatting_career(mask_result)

print(formatting_result)
